In [8]:
import sys
import pandas as pd
from dotenv import load_dotenv
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# require "HUGGINGFACEHUB_API_TOKEN" in .env file!
load_dotenv()

sys.path.append("../")
from utils.data_utils import preprocess_score_df, preprocess_pattern_map_df
from utils.data_structurer import DataStructurer
from utils.constants import ANSWER_COL

In [9]:
score_df = pd.read_csv("../data/csa_score_1.csv")
pattern_map_df = pd.read_csv("../data/pattern_map.csv")
score_df = preprocess_score_df(score_df)
pattern_map_df = preprocess_pattern_map_df(pattern_map_df)

In [10]:
structurer = DataStructurer(score_df, pattern_map_df, 1122128, 10)

In [11]:
def preprocess(input_string):
    input_string = str(input_string)
    preprocessed_string = input_string.strip()
    preprocessed_string = preprocessed_string.strip("\n")
    return preprocessed_string

In [12]:
template = """Compute the overall score of the response based on the provided criteria. (Max score is {max_score})
###Criteria### 
{criteria}

###Answer### 
{answer}

###Score###
"""

template = preprocess(template)
prompt = PromptTemplate(template=template, input_variables=["criteria", "answer", "max_score"])

In [13]:
# llm = HuggingFaceHub(
#         # repo_id="google/flan-t5-base",
#         repo_id="google/flan-t5-large",
#         # repo_id="SeaLLMs/SeaLLM-7B-v2",
#         # repo_id="scb10x/typhoon-7b",
#         # repo_id="openthaigpt/openthaigpt-1.0.0-alpha-7b-chat-ckpt-hf"
#         # repo_id="migtissera/Tess-M-Creative-v1.0",
#         # repo_id="google/flan-t5-xl",
#         # model_kwargs={
#         #     "temperature": 0,
#         #     "max_length": 128,
#         # }
#     )

# llm_chain = LLMChain(
#     prompt=prompt,
#     llm=llm,
# )

In [14]:
# criteria = """
# + 2 point for having risk mitigation.
# + 3 point for having data-driven decision.
# """
# answer = """Data-Driven Decisions and Risk Mitigation
# """
# max_score = 5
# # criteria = preprocess(criteria)
# # answer = preprocess(answer)

# # result = llm_chain.run({"criteria": criteria, "answer": answer})
# # print(result)


# def predict_score(answer, pattern, max_score):
#     criteria = preprocess(pattern) 
#     answer = preprocess(answer)

#     result = llm_chain.run({"criteria": criteria, "answer": answer, "max_score": max_score})
    
#     try:
#         result = float(result)
#     except ValueError:
#         print(result)
#         result = -1
#     return result

# predict_score(answer, criteria, max_score)

In [16]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="LoneStriker/SeaLLM-7B-v2-GGUF",
    task="text-generation",
    # pipeline_kwargs={"max_new_tokens": 10},
    device=1,
)

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "LoneStriker/SeaLLM-7B-v2-GGUF"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

chain = prompt | hf

def predict_score(answer, pattern, max_score):
    criteria = preprocess(pattern) 
    answer = preprocess(answer)

    result = chain.invoke({"criteria": criteria, "answer": answer, "max_score": max_score})
    
    try:
        result = float(result)
    except ValueError:
        print(result)
        result = -1
    return result

OSError: LoneStriker/SeaLLM-7B-v2-GGUF does not appear to have a file named config.json. Checkout 'https://huggingface.co/LoneStriker/SeaLLM-7B-v2-GGUF/main' for available files.

In [ ]:
predict_list = []
for index, row, in structurer.score_df.iterrows():
    pred_score_list = []
    answer = row[ANSWER_COL]
    
    for pattern_idx, (pattern, pattern_max_score) in enumerate(zip(structurer.pattern_list, structurer.pattern_max_score_list)):
        pred_score = predict_score(answer, pattern, pattern_max_score)
        pred_score_list.append(pred_score)
        # print(answer, pattern, pattern_max_score)
        print(f"{index=}, {pattern_idx=}\t, Actual: {row[f'pattern1_{pattern_idx+1}']}, Pred: {pred_score}, Max: {pattern_max_score}")
    predict_list.append(pred_score_list)
        
# structurer.pattern_list
predict_list

index=0, pattern_idx=0	, Actual: 0.0, Pred: 1.0, Max: 2.0
index=0, pattern_idx=1	, Actual: 0.0, Pred: 1.0, Max: 1.0
index=0, pattern_idx=2	, Actual: 1.0, Pred: 1.0, Max: 1.0
index=0, pattern_idx=3	, Actual: 1.0, Pred: 1.0, Max: 1.0
index=1, pattern_idx=0	, Actual: 0.0, Pred: 1.0, Max: 2.0
index=1, pattern_idx=1	, Actual: 0.0, Pred: 1.0, Max: 1.0
index=1, pattern_idx=2	, Actual: 0.0, Pred: 0.0, Max: 1.0
index=1, pattern_idx=3	, Actual: 1.0, Pred: 0.0, Max: 1.0
index=2, pattern_idx=0	, Actual: 2.0, Pred: 1.0, Max: 2.0
index=2, pattern_idx=1	, Actual: 1.0, Pred: 1.0, Max: 1.0
index=2, pattern_idx=2	, Actual: 1.0, Pred: 1.0, Max: 1.0
index=2, pattern_idx=3	, Actual: 1.0, Pred: 1.0, Max: 1.0
index=3, pattern_idx=0	, Actual: 2.0, Pred: 0.0, Max: 2.0
index=3, pattern_idx=1	, Actual: 0.0, Pred: 1.0, Max: 1.0
index=3, pattern_idx=2	, Actual: 1.0, Pred: 1.0, Max: 1.0
index=3, pattern_idx=3	, Actual: 0.0, Pred: 1.0, Max: 1.0
index=4, pattern_idx=0	, Actual: 2.0, Pred: 1.0, Max: 2.0
index=4, patte

ValueError: Error raised by inference API: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate